<a href="https://colab.research.google.com/github/Vathsav56/AI-Agent-Python/blob/main/code/ProjectWiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install wikipedia transformers pandas langdetect sentencepiece --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [10]:
from transformers import pipeline
import wikipedia
import pandas as pd
from langdetect import detect, LangDetectException
!pip install diffusers
import torch
from diffusers import DiffusionPipeline
import os


gen = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
# gen.to("cuda")

summarizer = None
try:
    summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")
except Exception:
    try:
        summarizer = pipeline("summarization", model="t5-small")
    except Exception:
        pass

translator_en_to_target = None
try:
    translator_en_to_target = pipeline("translation", model="Helsinki-NLP/opus-mt-en-hi")
except Exception:
    pass

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [2]:
def get_user_input():
    topic = input("Enter the topic you want to search on WEB: ")
    summarize_choice = input("Do you want to summarize the data? (yes/no): ").lower()
    translate_choice = input("Do you want to translate the data? (yes/no): ").lower()
    target_language = None
    if translate_choice == 'yes':
        target_language = input("Enter the target language code for translation (e.g., 'hi' for Hindi, 'fr' for French, 'es' for Spanish): ").lower()
    return topic, summarize_choice, translate_choice, target_language

In [3]:
def get_wikipedia_data(topic):
    try:
        print(f"\nSearching Wikipedia for: '{topic}'...")
        wiki_summary = wikipedia.summary(topic, sentences=10)
        print("Wikipedia summary retrieved.")
        return wiki_summary
    except wikipedia.exceptions.PageError:
        print(f"Error: Wikipedia page for '{topic}' not found.")
        return None
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation Error: '{topic}' could refer to multiple things. Suggestions: {e.options}")
        choice = input("Please be more specific or choose one from the suggestions (enter full suggestion): ")
        return get_wikipedia_data(choice)
    except Exception as e:
        print(f"An unexpected error occurred while fetching Wikipedia data: {e}")
        return None

In [4]:
def process_data(text, summarize_choice, translate_choice, target_language):
    processed_data = {"Original Text": text}

    if summarize_choice == 'yes' and summarizer:
        print("\nSummarizing data...")
        try:
            summary = summarizer(text, min_length=50, max_length=200)[0]['summary_text']
            processed_data["Summarized Text"] = summary
            print("Data summarized.")
        except Exception as e:
            print(f"Error during summarization: {e}")
            processed_data["Summarized Text"] = "Summarization failed."
    elif summarize_choice == 'yes' and not summarizer:
        processed_data["Summarized Text"] = "Summarizer model not loaded, skipping summarization."

    if translate_choice == 'yes' and translator_en_to_target:
        print(f"\nDetecting language of the original text for translation...")
        try:
            detected_lang = detect(text)
            print(f"Detected language: {detected_lang}")

            if detected_lang == 'en':
                print(f"Translating data to {target_language}...")
                try:
                    translation_result = translator_en_to_target(text)[0]['translation_text']
                    processed_data[f"Translated Text ({target_language})"] = translation_result
                    print("Data translated.")
                except Exception as e:
                    print(f"Error during translation: {e}")
                    processed_data[f"Translated Text ({target_language})"] = "Translation failed."
            else:
                print(f"Skipping translation: Translator is set for English source, but detected language is {detected_lang}.")
                processed_data[f"Translated Text ({target_language})"] = f"Skipped (source language not English)."
        except LangDetectException as e:
            print(f"Error detecting language: {e}. Skipping translation.")
            processed_data[f"Translated Text ({target_language})"] = "Language detection failed, skipping translation."
    elif translate_choice == 'yes' and not translator_en_to_target:
        processed_data[f"Translated Text ({target_language})"] = "Translation model not loaded, skipping translation."

    return processed_data

In [5]:
def generate_output_file(data, topic):
    file_name = f"{topic.replace(' ', '_')}_data.txt"
    try:
        with open(file_name, 'w', encoding='utf-8') as f:
            for key, value in data.items():
                f.write(f"--- {key} ---\n")
                f.write(value + "\n\n")
        print(f"\nData successfully saved to '{file_name}'")
    except Exception as e:
        print(f"Error saving data to file: {e}")

    csv_file_name = f"{topic.replace(' ', '_')}_data.csv"
    try:
        df = pd.DataFrame([data])
        df.to_csv(csv_file_name, index=False, encoding='utf-8')
        print(f"Data also saved to '{csv_file_name}' as CSV.")
    except Exception as e:
        print(f"Error saving data to CSV: {e}")

In [7]:
topic, summarize_choice, translate_choice, target_language = get_user_input()

wikipedia_text = get_wikipedia_data(topic)

if wikipedia_text:
    processed_data = process_data(wikipedia_text, summarize_choice, translate_choice, target_language)
    generate_output_file(processed_data, topic)
    ig = int(input("Would u like to generate a img (0/1): "))
    if ig:
      img = gen(topic).images[0]
      display(img)
    else:
      print("No image generated")
else:
    print("No data collected from Wikipedia. Exiting.")

Enter the topic you want to search on WEB: nvidia 5090
Do you want to summarize the data? (yes/no): yes
Do you want to translate the data? (yes/no): no

Searching Wikipedia for: 'nvidia 5090'...


NameError: name 'wikipedia' is not defined

In [ ]:

import os

file_name = f"{topic.replace(' ', '_')}_data.txt"

if os.path.exists(file_name):
  print(f"\n--- Content of '{file_name}' ---")
  with open(file_name, 'r', encoding='utf-8') as f:
    print(f.read())
else:
  print(f"File '{file_name}' was not found.")

csv_file_name = f"{topic.replace(' ', '_')}_data.csv"

if os.path.exists(csv_file_name):
    print(f"\n--- Content of '{csv_file_name}' ---")
    df_csv = pd.read_csv(csv_file_name)
    display(df_csv)
else:
    print(f"CSV file '{csv_file_name}' was not found.")